In [1]:
from scipy.stats import f as f_dist
import numpy as np


In [2]:
# create function to quickly sum squares within ANOVA test
def sum_squares(a):
    s = 0
    for i in range(len(a) - 1): 
        s += a[i]**2
    return s

# and one to square sums
def square_sums(a):
    s = 0
    for i in range(len(a) - 1): 
        s += a[i]
    return s * s

# finds value of f-distribution for given f and df1, df2
def f_prob(f, df1, df2):
    # use scipy to plug f-value into f distribution to return p-value
    p_value = f_dist.sf(df1, df2, f)
    # could attempt to manually implement, i.e
    # [f(x, df_1, df_2) = (df_2^{df_2/2} df_1^{df_1/2} x^{df_1 / 2-1} / 
    #                   {(df_2+df_1 x)^{(df_1+df_2)/2}*sc.beta(df_1/2, df_2/2)}\]
    return p_value

In [3]:
# use *args command to accept variable number of arguments
def ANOVA(*args):
    # Create array of arguments using generator function
    args = [np.asarray(arg) for arg in args]
    # ANOVA on N groups, each in its own array
    k = len(args)
    print("k: ", k)
    alldata = np.concatenate(args)
    bign = 0
    for i in range(k):
        bign += len(args[i])
    
    print("n: ", bign)

    # Determine the grand mean of the data, and subtract that from all inputs
    grand_mean = alldata.mean()
    print("grand mean: ", grand_mean)

    sstot = 0
    for i in range(bign):
        sstot += (alldata[i] - grand_mean)**2
    print("sstot: ", sstot)
    
    # find sum of squares within data by adding together the square of the difference between each datapoint in 
    # the kth group minus the mean of the kth group 
    sswn = 0
    for i in range(k):
        #for j in range(len(args[i])):
            sswn += sum_squares(args[i] - args[i].mean())
    print("sswn: ", sswn)
    
    # find sum of squares between data by adding together the square of the difference between the kth group mean
    # and the grand mean
    ssbn = 0
    for i in range(k):  
        ssbn += (args[i].mean() - grand_mean)**2
    print("ssbn: ",ssbn)

    # variables ending in bn/b are for "between treatments", wn/w are
    # for "within treatments"
    dfbn = k - 1
    dfwn = bign - k
    print("k: ", k, "n: ", bign)
    print("DFbn, DFwn: ", dfbn,",", dfwn)
    msb = ssbn / float(dfbn)
    print("msb: ", msb)
    msw = sswn / float(dfwn)
    print("msw: ", msw)
    f = msb / msw

    p_value = float(f_prob(f, dfbn, dfwn))

    if p_value > .05:
        return_string = "F-value: " + str(f) + ", P-value: " + str(
            p_value) + ", Fail to reject null hypothesis."
    else:
        return_string = "F-value: " + str(f) + ", P-value: " + str(
            round(p_value, 5)) + ", Reject null hypothesis."

    return return_string

In [4]:
# test runs
data1 = np.random.normal(size=20)
data2 = np.random.normal(size=20)
data3 = np.random.normal(size=20)

ANOVA(data1, data2, data3)

k:  3
n:  60
grand mean:  -0.19154522862059895
sstot:  44.57599800129165
sswn:  42.933693073281056
ssbn:  0.012601218968706217
k:  3 n:  60
DFbn, DFwn:  2 , 57
msb:  0.006300609484353108
msw:  0.7532226854961589


'F-value: 0.00836486952089448, P-value: 0.9767856470350209, Fail to reject null hypothesis.'

In [9]:
d1 = np.random.normal(loc = 10, size=100)
d2 = np.random.normal(loc = 3, size=100)
d3 = np.random.normal(loc = 2, size=100)

ANOVA(d1, d2, d3)

k:  3
n:  300
grand mean:  5.037572836392548
sstot:  4150.079585712167
sswn:  326.93313152887333
ssbn:  38.17735620761645
k:  3 n:  300
DFbn, DFwn:  2 , 297
msb:  19.088678103808224
msw:  1.1007849546426711


'F-value: 17.340969299498333, P-value: 0.04597, Reject null hypothesis.'

In [10]:
import scipy.stats as st 
st.f_oneway(d1, d2, d3)


F_onewayResult(statistic=1705.8644039692633, pvalue=1.494199824158835e-163)